<img src="https://www3.um.edu.uy/logoum.jpg" width=300>
<h1 align="center">Thesis - Funk SVD MAE</h1> 
<h2 align="center">Alejo Paullier</h2> 

https://lkpy.lenskit.org/en/stable/knn.html

In [1]:
from lenskit import batch, topn, util
from lenskit import crossfold as xf
from lenskit.algorithms import Recommender, funksvd
from lenskit.algorithms.funksvd import FunkSVD
from lenskit.metrics.predict import mae
import pandas as pd
import numpy as np

ratings = pd.read_csv('C:\\Users\\Alejo\\Tesis\\Demo\\ml-100k\\u.data', sep='\t',
                      names=['user', 'item', 'rating', 'timestamp'])


def eval(aname, algo, train, test):
    fittable = util.clone(algo) # Object cloning means to create an exact copy of the original object.
    fittable = Recommender.adapt(fittable) 
    fittable.fit(train) # train the algorithm with the training dataset
    # Now we run the recommender:
    preds = batch.predict(fittable, test)
    # add the algorithm name for analyzability
    return preds

In [57]:
def model_trainer(features,lrate,reg):
    all_recs = []
    test_data = []
    features=int(features)
    funksvd = FunkSVD(features=features, iterations=100, lrate=lrate, reg=reg, damping=5, range=None, bias=True) # define algorithm
    
    for train, test in xf.partition_users(ratings[['user', 'item', 'rating']], 5, xf.SampleFrac(0.2)):
        test_data.append(test) # save testing data
        all_recs.append(eval('FunkSVD', funksvd, train, test))
    
    all_recs = pd.concat(all_recs, ignore_index=True)

    metric = mae(all_recs['prediction'], all_recs['rating'])
    return metric

In [58]:
from hyperopt import fmin, tpe, hp, STATUS_OK

def objective(params):
    features = params['features']
    lrate = params['lrate']
    reg = params['reg']
    metric = model_trainer(features,lrate,reg)
    print([features,lrate,reg,metric],',')
    return {'loss': metric, 'status': STATUS_OK }

space={'features': hp.uniform('features', 1, 100),
       'lrate': hp.uniform('lrate', 0.0005, 0.005),
       'reg': hp.uniform('reg',0.005,0.05)}

best = fmin(objective, space, algo=tpe.suggest,max_evals=100)

[24.251151391291977, 0.001638043152304595, 0.021823854005600423, 0.7221333261074406]                                   
,                                                                                                                      
[29.091714253932142, 0.0016346273633924939, 0.027997933722311, 0.7306106951074655]                                     
,                                                                                                                      
[98.81137976808395, 0.00475318484233948, 0.04667847037885562, 0.7288232928678229]                                      
,                                                                                                                      
[32.23183805728489, 0.003380694291735812, 0.018214288644447896, 0.7419923774331019]                                    
,                                                                                                                      
[41.60071842219373, 0.002226526932594864

[60.3507001826729, 0.0028652070916747032, 0.023501142373171677, 0.7266568305547695]                                    
,                                                                                                                      
[55.10506201686649, 0.002115215676985082, 0.026792723528357678, 0.7113568048739165]                                    
,                                                                                                                      
[38.66566341570888, 0.0015997103863998265, 0.025852827574265578, 0.7256052018803667]                                   
,                                                                                                                      
[69.61092625043845, 0.0024681586378683742, 0.020551480313404896, 0.7170620835598306]                                   
,                                                                                                                      
[74.62570446564456, 0.001888141697184280

In [60]:
metric = pd.DataFrame(array)
metric.columns = ['features','learning rate','reg','MAE']
metric.head(5)

,features,learning rate,reg,MAE
0,24.251151,0.001638,0.021824,0.722133
1,29.091714,0.001635,0.027998,0.730611
2,98.811380,0.004753,0.046678,0.728823
3,32.231838,0.003381,0.018214,0.741992
4,41.600718,0.002227,0.030675,0.704274


In [61]:
metric.sort_values(by=['MAE'],ascending=True).head(10)

,features,learning rate,reg,MAE
35,68.745136,0.002374,0.032139,0.703654
4,41.600718,0.002227,0.030675,0.704274
84,20.191007,0.002346,0.021524,0.708949
89,5.162615,0.003383,0.039766,0.709153
40,60.902272,0.002419,0.023376,0.710706
83,52.870145,0.002513,0.043243,0.710747
69,55.105062,0.002115,0.026793,0.711357
67,51.550824,0.002452,0.030005,0.711398
25,72.286529,0.002164,0.030665,0.711976
66,62.242457,0.002254,0.022171,0.712353


In [62]:
metric.to_csv('metric_MAE.csv', index=False)

In [2]:
metric = pd.read_csv('C:\\Users\\Alejo\\Tesis\\Algoritmos\\FunkSVD\\results\\metric_MAE.csv')

In [3]:
%matplotlib qt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm
colormap = cm.hot
x = metric['reg']
y = metric['learning rate']
z = metric['features'] 
c = metric['MAE']


fig = plt.figure()

ax = fig.add_subplot(111, projection='3d')
img = ax.scatter(x, y, z, c=c, cmap=plt.hsv())
ax.set_xlabel('reg')
ax.set_ylabel('learning rate')
ax.set_zlabel('features')
fig.colorbar(img)
plt.show()